<a href="https://colab.research.google.com/github/jeffersonramelo/Textual-Analysis/blob/main/Sentimento_FINBERT_COPOM_EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
# Carregar os dados do dataframe
dados = pd.read_excel("/content/atas_copom_texto_limpo.xlsx") 


In [ ]:
dados.head()

,DataReferencia,Titulo,Unnamed: 2,texto,data,Meta SELIC,sentimento_taxa,ata
0,2023-05-03T03:00:00Z,"254th Meeting - May 2-3, 2023",NaN,banco central brasil focus january minutes nd ...,1999-12-15,19.0,incerto,42
1,2020-08-05T03:00:00Z,"232nd Meeting – August 4-5, 2020",NaN,banco central brasil focus february minutes nd...,2000-01-19,19.0,incerto,43
2,2016-05-11T14:59:38Z,198th Copom Minutes,NaN,xppdu jjuhjdwhvxssodqgghpdqg whuqdohqylurqphq...,2000-02-16,19.0,incerto,44
3,2015-11-25T02:00:00Z,195th Copom Minutes,NaN,banco central brasil focus march minutes th me...,NaT,18.5,positivo,45
4,2014-12-17T19:19:49Z,187th Copom Minutes,NaN,banco central brasil focus march minutes th me...,2000-03-22,19.0,incerto,45


In [ ]:
textos = dados["texto"].tolist()

In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.2 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Carregar o modelo FinBERT pré-treinado
model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Pré-processamento e análise de sentimento
sentimentos = []

for texto in textos:
    encoded_input = tokenizer(texto, padding=True, truncation=True, max_length=512, return_tensors="pt")
    output = model(**encoded_input)
    logits = output.logits
    predicted_label = torch.argmax(logits).item()
    sentimentos.append(predicted_label)

# Mapear os valores de sentimento para rótulos correspondentes
mapeamento_sentimento = {0: "negativo", 1: "neutro", 2: "positivo"}
sentimentos = [mapeamento_sentimento[sentimento] for sentimento in sentimentos]


# Criar nova coluna com a classificação de sentimento
dados["sentimentoFINBERT"] = sentimentos




In [ ]:
# Visualizar os resultados
dados.head()

,DataReferencia,Titulo,Unnamed: 2,texto,data,Meta SELIC,sentimento_taxa,ata,sentimentoFINBERT
0,2023-05-03T03:00:00Z,"254th Meeting - May 2-3, 2023",NaN,banco central brasil focus january minutes nd ...,1999-12-15,19.0,incerto,42,positivo
1,2020-08-05T03:00:00Z,"232nd Meeting – August 4-5, 2020",NaN,banco central brasil focus february minutes nd...,2000-01-19,19.0,incerto,43,positivo
2,2016-05-11T14:59:38Z,198th Copom Minutes,NaN,xppdu jjuhjdwhvxssodqgghpdqg whuqdohqylurqphq...,2000-02-16,19.0,incerto,44,positivo
3,2015-11-25T02:00:00Z,195th Copom Minutes,NaN,banco central brasil focus march minutes th me...,NaT,18.5,positivo,45,positivo
4,2014-12-17T19:19:49Z,187th Copom Minutes,NaN,banco central brasil focus march minutes th me...,2000-03-22,19.0,incerto,45,positivo


In [ ]:
# Contar a frequência de cada valor na coluna "sentimentoFINBERT"
contagem_sentimentos = dados["sentimentoFINBERT"].value_counts()

# Exibir os resultados
print(contagem_sentimentos)


positivo    200
neutro       14
Name: sentimentoFINBERT, dtype: int64
